# GM(1,1)

## 原始数据检测
### 是否有负数？是否有周期规律？
画图呈现原始数据
### 期数是否高于4期？是否低于10期？
列表+画图呈现原始数据

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# 指定文件路径
file_path = '指定路径/date.xlsx'
# 使用pandas读取Excel文件
myfile = pd.read_excel(file_path)
# 选择第一列作为t，第二列作为d
t = myfile.iloc[:, 0]  # 选择第一列
date = myfile.iloc[:, 1]  # 选择第二列

# 画折线图
plt.plot(t, date, label='折线图')
# 添加标题和轴标签
plt.title('折线图')
plt.xlabel('t')
plt.ylabel('d')
# 添加图例
plt.legend()
# 显示图形
plt.show()

### 准指数规律检验
指标1：光滑比小于0.5的数据占比（一般要大于60%）
指标2：除去前两个时期外，光滑比小于0.5的数据占比（一般大于90%）

In [ ]:
X0 = np.array(date)
X1 = X0.cumsum()
rho = [X0[i] / X1[i - 1] for i in range(1,len(X0))]
rho_ratio = [rho[i + 1] / rho[i] for i in range(len(rho) - 1)]
threshold = len(rho)*0.6
print("rho:",rho)
print("rho_ratio:",rho_ratio)
count_out = 0
for i in range(1,len(rho) - 1):
    if rho[i] > 0.5 or rho[i + 1] / rho[i] >= 1:
        count_out += 1
if rho[-1] > 0.5:
    count_out += 1
if count_out < threshold:
    print("数据通过光滑校验")
else:
    print("该数据未通过光滑校验")

级比检验

In [ ]:
import math as mt
for i in range(len(X0)-1):
    l = X0[i]/X0[i+1]
    if l <= mt.exp(-2/(len(X0)+1)) or l >= mt.exp(2/(len(X0)+1)):
        break
    else:
        pass
if i == len(X0)-2 and l > mt.exp(-2/(len(X0)+1)) and l < mt.exp(2/(len(X0)+1)):
    print('级比检验通过')
else:
    print('级比检验不通过')


级比检验不通过处理

In [ ]:
import math as mt
j = 1
while True:
    YO = [k+j for k in X0]
    j += 1
    for m in range(len(YO) - 1):
        l = YO[m] / YO[m + 1]
        if l > mt.exp(-2 / (len(X0) + 1)) and l < mt.exp(2 / (len(X0) + 1)):
            b = True
        else:
            b = False
            break
    if b == True:
        print("新的原始数列为：",YO)
        c = j -1
        print("c的值为：",c)
        break
    else:
        continue

## 模型训练和测试
### 数据期数特别少
分别计算出传统的GM(1,1)模型、新信息GM(1,1)模型和新陈代谢GM(1,1)模型对于未来期数的预测结果，为了保证结果的稳健性，对三个结果求平均值作为预测值。

In [ ]:
Z = []
j = 1
while j < len(X1):
    num = (X1[j] + X1[j - 1]) / 2
    Z.append(num)
    j = j + 1

# 最小二乘法计算
Y = []
x_i = 0
while x_i < len(X0) - 1:
    x_i += 1
    Y.append(X0[x_i])
Y = np.mat(Y)
Y = Y.reshape(-1,1)
B = []
b = 0
while b < len(Z):
    B.append(-Z[b])
    b += 1
B = np.mat(B)
B = B.reshape(-1,1)
c = np.ones((len(B),1))
B = np.hstack((B,c))
print("B",B)

# 求出参数
alpha = np.linalg.inv(B.T.dot(B)).dot(B.T).dot(Y)
a = alpha[0,0]
b = alpha[1,0]
print('alpha',alpha)
print("a=",a)
print("b=",b)

# 生成预测模型
GM = []
GM.append(X0[0])
did = b/a
k = 1
while k < len(X0):
    GM.append((X0[0] - did) * mt.exp(-a * k) + did)
    k += 1

# 做差得到预测序列
G = []
G.append(X0[0])
g = 1
while g < len(X0):
    G.append(round(GM[g] - GM[g - 1]))
    g += 1
print("预测数列为：",G)

### 数据期数较多
取最后` k (k>1)如果k==1，则三个模型结果相等`期为试验组，前面的` n-k `期为训练组；
用训练组的数据分别训练三种GM模型，将训练出来的模型分别用于预测试验组的两期数据
利用试验组` k `期的真实数据和预测出来的两期数据，可分别计算出三个模型的SSE；选择SSE最小的模型作为我们建模的模型。

## 评价模型
输出并绘制图形显示预测结果，并进行残差检验和级比偏差检验。


In [ ]:
import matplotlib.pyplot  as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False 

X0 = ['原始数据']
G = ['预测数据']
G = [g-37315 for g in G]
r = range(len(X0))
t = list(r)
plt.plot(t,X0,color='r',linestyle="--",label='true')
plt.plot(t,G,color='b',linestyle="--",label="predict")
plt.legend()
plt.show()


模型检验

In [ ]:
import numpy as np
import math as mt
X0 = ['原始数据']
G = ['预测数据']
X0 = np.array(X0)
G = np.array(G)
e = X0 - G  #残差
q = e / X0  # 相对误差
w = 0
for q_i in q:
    w += q_i
w = w/len(X0)
print('精度为{}%'.format(round((1-w)*100,2)))

s0 = np.var(X0)
s1 = np.var(e)
S0 = mt.sqrt(s0)
S1 = mt.sqrt(s1)
C = S1 / S0
print('方差比为：',C)

p = 0
for s in range(len(e)):
    if (abs(e[s]-np.mean(e)) < 0.6745 * S1):
        p = p + 1
P = p / len(e)
print('小概率误差为：',P)
